In [112]:
pip install --user opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [113]:
pip install opencv-python opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [114]:
import cv2
import numpy as np
import csv

# Function to convert a rotation matrix to Euler angles
def rotationMatrixToEulerAngles(R):
    # Calculate the sy value
    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    # Check for singularity
    singular = sy < 1e-6

    if not singular:
        # Calculate the angles
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        # Handle the singularity case
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    # Convert the angles from radians to degrees
    return np.degrees(x), np.degrees(y), np.degrees(z)

In [115]:

# Load the ArUco dictionary that contains the markers we will detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
# Create detector parameters
parameters = cv2.aruco.DetectorParameters()

In [116]:
#camera calibaration by boaz:
camera_matrix = np.array([[921.170702, 0.000000, 459.904354], [0.000000, 919.018377, 351.238301], [0.000000, 0.000000, 1.000000]])
distortion = np.array([-0.033458, 0.105152, 0.001256, -0.006647, 0.000000])

In [117]:

# Define the size of the ArUco marker in meters
marker_length = 0.14  # aruco marker size

# Initialize video capture from the provided video file
cap = cv2.VideoCapture('challengeB.mp4')

# Open a CSV file to write the results
csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
# Write the header row
csv_writer.writerow(['Frame ID', 'QR id', 'QR 2D', 'QR 3D'])

frame_id = 0

# Initialize the video writer for the annotated output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')

'''
Params of VideoWriter:
video output name , codec , frame rate - FPS , width of the frames , height of the frames
'''
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

In [118]:
import time

frames_process_time_sum = 0

# Process the video frame by frame
while cap.isOpened():
    
    # Start time for measuring frame processing time
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect ArUco markers in the frame
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    
    if ids is not None:
        # Estimate the pose of each detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)
        
        detected_ids = []

        for i in range(len(ids)):
            # Get the rotation and translation vectors for each marker
            rvec, tvec = rvecs[i][0], tvecs[i][0]
            id = ids[i][0]
            detected_ids.append(id)
            # Get the 2D corner points of the marker
            corner = corners[i].reshape(4, 2).tolist()
            
            # Draw the detected markers on the frame
            cv2.aruco.drawDetectedMarkers(frame, corners)
            # Draw the axes of the detected marker
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, marker_length * 0.5)
            
            # Convert the rotation vector to a rotation matrix
            R, _ = cv2.Rodrigues(rvec)
            # Calculate the yaw, pitch, and roll angles
            yaw, pitch, roll = rotationMatrixToEulerAngles(R)
            
            '''
            Distance: the distance from the camera to the ArUco marker, measured in meters. This is the Euclidean norm of the translation vector tvec.

            Yaw: the yaw angle of the ArUco marker, measured in degrees. Yaw represents the rotation around the vertical axis (often the Z-axis in the camera's coordinate system).

            Pitch: the pitch angle of the ArUco marker, measured in degrees. Pitch represents the rotation around the lateral axis (often the X-axis in the camera's coordinate system).

            Roll: the roll angle of the ArUco marker, measured in degrees. Roll represents the rotation around the longitudinal axis (often the Y-axis in the camera's coordinate system).
            '''
            
            # Write the data to the CSV file
            csv_writer.writerow([frame_id, id, corner, [np.linalg.norm(tvec), yaw, pitch, roll]])
        
        # Draw the IDs of the detected markers in the center of the screen
        ids_text = "Detected IDs: " + ", ".join(map(str, detected_ids))
        screen_center_x = frame.shape[1] // 2
        screen_center_y = frame.shape[0] // 2
        text_pos = (screen_center_x - 100, screen_center_y)  # Adjust as necessary for better positioning
        cv2.putText(frame, ids_text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Write the annotated frame to the output video
    out.write(frame)
    
    
    # End time for measuring frame processing time
    end_time = time.time()
    elapsed_time_ms = (end_time - start_time) * 1000.0
    #print(f"Frame {frame_id}: {elapsed_time_ms:.2f} ms")

    
    #summing up the time of processing all frames for calculating an average:
    frames_process_time_sum += elapsed_time_ms
    

    # Check if processing time exceeds 30ms
    #if elapsed_time_ms > 30.0:
        #print(f"WARNING: Frame {frame_id} took {elapsed_time_ms:.2f} ms to process, which exceeds 30 ms.")
    
    
    frame_id += 1

    
    # Check for user input to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
average_time_to_process_a_frame = float(frames_process_time_sum) / float(frame_id)
print(f"average time it took to process a single frame: {average_time_to_process_a_frame:.2f}")        

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()
# Close the CSV file
csv_file.close()
#print("done")


average time it took to process a single frame: 13.39


In [119]:
# wait for printing of the average time it took to process a single frame